In [16]:
import tensorly as tl
from tensorly.decomposition import symmetric_power_iteration
import numpy as np
np.set_printoptions(precision=3, suppress=True)

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
  
def svd_whiten(X, k):
    # X: matrix to decompose
    # k: rank approximation
    
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    # rank k approximation
    U = U[:,:k]
    s = s[:k]
    Vt = Vt[:k,:]
    
    D_half = np.diag(np.sqrt(s))
    W = np.diag(1./np.sqrt(s)) @ U.T

    # so np.dot(np.dot(W.T, X),W) is the identity matrix
        
    return W , D_half, U

def rank_k_pseudoinverse(X, k):
    # pseudoinverse of rank k approximation of X
    U, s, Vt = np.linalg.svd(X, full_matrices=False)

    return np.linalg.pinv(U[:,:k] @ np.diag(s[:k]) @ Vt[:k,:])

def model_1():
    component_param =  {0: np.array([[0.15, 0.05, 0.8],
                                    [0.2, 0.15, 0.65],
                                    [0.12, 0.14, 0.74]]),
                      1: np.array([[0.2, 0.1  , 0.7],
                                   [0.1, 0.1, 0.8],
                                   [0.3, 0.2,  0.5]]),
                      2: np.array([[0.4, 0.15, 0.45],
                             [0.9, 0.03, 0.07],
                             [0.1, 0.7, 0.2]])}
    
    h_weights = np.array([0.2, 0.5, 0.3])
    return component_param, h_weights

def model_2():
    component_param =  {0: np.array([[0.15, 0.05, 0.2, .01, .59],
                                    [0.2, 0.15, .04, 0.55, .06],
                                    [0.12, 0.14, 0.7, .02, .02]]),
                      1: np.array([[0.2, 0.4  , 0.2, .1, .1],
                                   [0.1, 0.03, 0.57, .15, .15],
                                   [0.3, 0.1, .1, .33, .17]]),
                      2: np.array([[0.4, 0.45, .03, .02, .1],
                             [0.1, .22, .58, 0.03, 0.07],
                             [0.12, 0.38, .18, .12, .2]])}
    
    h_weights = np.array([0.2, 0.5, 0.3])
    return component_param, h_weights


def generate_data_fixed(data_size=10000):
    # num_components = num_rows = num_views = 3
    
    component_param, h_weights = model_2()
    
    comp_label = np.arange(0,len(h_weights))  # there are num_components of components; zero-indexing
    observation_label = np.arange(0, len(component_param[0][0]))
    
    data = []
    for i in range(data_size):
        # choose a component
        choose = np.random.choice(comp_label, p = h_weights)
        datum = []
        for v in range(3):
            datum.append(np.random.choice(observation_label, p = component_param[choose][v]))            
        data.append(tuple(datum))
        
    rank = len(h_weights)
    obs_size = len(component_param[0][0])
    return  rank, obs_size, component_param, h_weights, data

### Generate data

In [28]:
data_len = 15000
rank, obs_size, cp, h, data = generate_data_fixed(data_size=data_len)
identity = np.eye(obs_size)
get_identity_col = lambda col_num: identity[:, col_num]
e0 = get_identity_col(0); e1 = get_identity_col(1); e2 = get_identity_col(2);

def create_view_vecs(data, view):
    return np.column_stack([get_identity_col(datum[view]) for datum in data])

### Calculate second moments and prepare matrices for view symmetrization

In [29]:
view0s = create_view_vecs(data,0)
view1s = create_view_vecs(data,1)
view2s = create_view_vecs(data,2)

M01 = (view0s @ view1s.T)/len(data)
M02 = (view0s @ view2s.T)/len(data)
M10 = (view1s @ view0s.T)/len(data)
M12 = (view1s @ view2s.T)/len(data) 
M20 = (view2s @ view0s.T)/len(data)
M21 = (view2s @ view1s.T)/len(data)


M01_pinv = np.linalg.pinv(M01)
M02_pinv = np.linalg.pinv(M02)
M10_pinv = np.linalg.pinv(M10)
M12_pinv = np.linalg.pinv(M12)
M20_pinv = np.linalg.pinv(M20)
M21_pinv = np.linalg.pinv(M21)


C_0_to_2 = M21 @ M01_pinv
C_1_to_2 = M20 @ M10_pinv
C_2_to_1 = M10 @ M20_pinv
C_2_to_0 = M01 @ M21_pinv

M_2 = 0.5* (C_0_to_2 @ M01 @ C_1_to_2.T + C_1_to_2 @ M10 @ C_0_to_2.T)
np.sum(M_2)

1.0000000000000016

### Calculate and symmetrize the third moment

In [30]:
def get_all_3rd_order_tensor(view0s, view1s, view2s, obs_size):
    # get all the dummy estimate (before view transform) for all 3rd-order tensors
    M3 = {(i,j,k):np.zeros((obs_size, obs_size, obs_size)) if i != j and i != k and j != k else None \
                                    for i in range(3)\
                                    for j in range(3)\
                                    for k in range(3)}
    M3 = {key:value for key, value in M3.items() if M3[key] is not None}
    views = [view0s, view1s, view2s]
    for key in M3.keys():
        for i in range(data_len):
            M3[key] += views[key[0]][:,i][:,None,None] * views[key[1]][:,i][None,:,None] * views[key[2]][:,i][None,None,:]
        M3[key] /= data_len
    return M3

M3 = get_all_3rd_order_tensor(view0s, view1s, view2s, obs_size)
M012_trans = tl.tenalg.multi_mode_dot(M3[0,1,2], [C_0_to_2, C_1_to_2], modes=[0,1])
M021_trans = tl.tenalg.multi_mode_dot(M3[0,2,1], [C_0_to_2, C_1_to_2], modes=[0,2])
M102_trans = tl.tenalg.multi_mode_dot(M3[1,0,2], [C_1_to_2, C_0_to_2], modes=[0,1])
M120_trans = tl.tenalg.multi_mode_dot(M3[1,2,0], [C_1_to_2, C_0_to_2], modes=[0,2])
M201_trans = tl.tenalg.multi_mode_dot(M3[2,0,1], [C_0_to_2, C_1_to_2], modes=[1,2])
M210_trans = tl.tenalg.multi_mode_dot(M3[2,1,0], [C_1_to_2, C_0_to_2], modes=[1,2])

M_3 = (M012_trans+M021_trans+M102_trans+M120_trans+M201_trans+M210_trans)/6
np.sum(M_3)

1.0000000000000016

### Whiten the tensors

In [31]:
W, D_half, U = svd_whiten(M_2, rank) # a bit a cheating here by using rank variable
matprint( W @ M_2 @ W.T)
whitened_t01 = tl.tenalg.multi_mode_dot(M_2, [W, W], modes=[0,1])
whitened_t012 = tl.tenalg.multi_mode_dot(M_3, [W, W, W], modes=[0,1,2])

           1  -2.99636e-16     1.677e-16  
-2.51893e-16             1  -6.05262e-17  
 1.48527e-16   3.00008e-16             1  


### Tensor power iteration (whitened case)

In [32]:
w = []
u = []
deflated = None
for i in range(3):
    if deflated is None:        
        w_temp, u_temp, deflated = symmetric_power_iteration(whitened_t012)        
    else:
        w_temp, u_temp, deflated = symmetric_power_iteration(deflated)
    w.append(w_temp)
    u.append(u_temp)

In [33]:
recovered_h = np.array([pow(1/i,2) for i in w])
os = [w[index] * U @ D_half @ u_i for index, u_i in enumerate(u)]
for index, os_i in enumerate(os):
    print("component weight: ", round(recovered_h[index], 3))
    print("first view: ", C_2_to_0 @ os_i )
    print("second view:", C_2_to_1 @ os_i )
    print("third view: ", os_i)
    print(" ")    

component weight:  0.179
first view:  [0.125 0.035 0.198 0.011 0.631]
second view: [0.222 0.156 0.05  0.55  0.022]
third view:  [0.092 0.141 0.738 0.02  0.009]
 
component weight:  0.294
first view:  [0.404 0.414 0.04  0.02  0.124]
second view: [0.094 0.264 0.496 0.04  0.107]
third view:  [0.107 0.411 0.164 0.123 0.196]
 
component weight:  0.518
first view:  [0.203 0.423 0.191 0.09  0.094]
second view: [0.099 0.01  0.595 0.156 0.14 ]
third view:  [0.303 0.095 0.114 0.312 0.176]
 


### Asymmetric tensor power iteration (non-whitened case)

In [34]:
M01_pinv = rank_k_pseudoinverse(M01, rank)
M02_pinv = rank_k_pseudoinverse(M02, rank)
M10_pinv = rank_k_pseudoinverse(M10, rank)
M12_pinv = rank_k_pseudoinverse(M12, rank)
M20_pinv = rank_k_pseudoinverse(M20, rank)
M21_pinv = rank_k_pseudoinverse(M21, rank)

T = M3[0,1,2]
for _ in range(rank):
    u = np.random.rand(obs_size)
    u = np.divide(u, np.linalg.norm(u))
  
    for _ in range(80):
        u = tl.tenalg.multi_mode_dot(T, [M01_pinv @ u, M02_pinv @ u], modes=[1,2])
        u = np.divide(u, np.linalg.norm(u))
    
    a = np.divide(u, np.sqrt(np.abs(np.dot(u, M10_pinv @ M12 @ M02_pinv @ u))))
    b = M12 @ M02_pinv @ a
    c = M21 @ M01_pinv @ a
    
    lambda_1 =  tl.tenalg.multi_mode_dot(T, [ M10_pinv @ M12 @ M02_pinv @ a, M01_pinv @ a, M02_pinv @ a], modes=[0,1,2])
    T = T - abs(lambda_1)*np.tensordot(np.tensordot(a,b,axes=0),c,axes=0) # deflation
    
    print("component weight: ", round(pow(1/lambda_1,2),3))
    print("first view: ", a/np.sum(a))
    print("second view:", b/np.sum(b))
    print("third view: ", c/np.sum(c))
    print(" ")

component weight:  0.194
first view:  [0.138 0.054 0.188 0.005 0.616]
second view: [0.218 0.157 0.025 0.545 0.055]
third view:  [0.091 0.158 0.724 0.005 0.022]
 
component weight:  0.515
first view:  [0.216 0.427 0.182 0.097 0.079]
second view: [0.089 0.033 0.594 0.126 0.158]
third view:  [0.295 0.097 0.091 0.327 0.19 ]
 
component weight:  0.31
first view:  [0.389 0.446 0.041 0.026 0.098]
second view: [0.097 0.224 0.591 0.03  0.058]
third view:  [0.133 0.411 0.135 0.136 0.185]
 
